In [2]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [3]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051

In [4]:
import os
from pyngrok import ngrok

In [5]:
ngrok.kill()

In [6]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://noncalculative-semireactionary-debra.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://noncalculative-semireactionary-debra.ngrok-free.dev


True

In [7]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [8]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [9]:
result = stateful_query("簡介勁戰六代(50字以內)")
print(result)

勁戰六代搭載水冷BLUE CORE引擎與非對稱車架，動力操控表現出色。具備ABS/TCS，提供更佳的運動性能與騎乘安全。


In [10]:
result2 = stateful_query("勁戰六代是什麼？(50字以內)")
print(result2)

Yamaha水冷運動速克達，搭載BLUE CORE引擎與ABS/TCS，提供優異性能及騎乘安全，是熱門的通勤休閒選擇。


In [11]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594452734154375291","quoteToken":"JGRoO8DNb2fcRIip8bWNGbF6UoqUcm1Z4RN44RssmUDKPZqTXV3wKPZdWwFfKAjVXKfBDg2yBirn8BTgWN-3FmrpIm5YiEFJjRXoYB3iy2FUrJH43861LU7_l4fHxgIWxmJbhMF5mw9rAKuY7JMtfg","markAsReadToken":"tsNGIl7fd4cddPSwqSbR_PErOeyiINt1UgmSWIJrAeaqAqkPBwej3D5CUVBP5sZYlO2hHT1gC60DKPzg5avzrUSiOpUk26UmtkvagWD586UnaHXxEJ-WrrLFAwWFt7SmRVpZI_-3-vMme6Zpwyu-rhlFgukWoTR3i_TiJXOq6rJ8KnrUZFrgYT1zBAaQvXRR8G8e-cPr9nZncCcZUumMlg","text":"AI 簡介勁戰六代（50字以內）"},"webhookEventId":"01KDS85YQ5X9YKMJVMAJBQVX3D","deliveryContext":{"isRedelivery":false},"timestamp":1767152744685,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"2598f4ed726d404d8d0ec678f22c1415","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 03:45:55] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594452793394987027","quoteToken":"ODDXTcaEbngmEOa_b6GrhZ-VvwePhcr1Z3tYLsgW2FE2bkXrlfaaNl6xXR47pcwD6w7oOC9Zn5YJ9Pxug9vwGKxaPnXhZHBKngczh3F3AFcw-EFicD-Rk5IqxBowtX6g9RXm-J30DCTcUGiaSBkW3A","markAsReadToken":"Ba6sBQiwF2Bdv9Bna6mTR7PUqxLK8_ELu0bsrEQQ8_ohHN-yUzjV8UbXddZNEcOWMW-VaPWXu2KuRvv3xS_WXtjNbiFPSxZG3z_UI2R8-G5_PjlWLc-z-Svptl0_nr4QOIGx1jPZAV-nbR6cCAVf9Tjn8OlwUKG3dCoiKYZ5_85mjgPPLCAButkuKxQrgfOwFBzKVp0XWJhz_Qa5a3LvQQ","text":"勁戰六代是什麼？（50字以內）"},"webhookEventId":"01KDS870Z84WJ9828YE43B6A36","deliveryContext":{"isRedelivery":false},"timestamp":1767152779914,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"3dcf3286bf024615b19c897754641b97","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 03:46:21] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594452842501112055","quoteToken":"ueqtS_YGK2vd40iSGVfdc1ktO4i6rBrGuNwoj1I-1Irwx5WdT81PXCmOCOWWWfFc9vlZC4lenU-UMqYoldapW2i0tA_8Jb8xOrr7iQcLSZR05GqRs5a2Xd90AS6lZWkAhT_Ot3xFiIQdbjFC8Htchw","markAsReadToken":"DDhwGk3QPz7zLETWogjHXuJS3pzDftd7mNeuD4r_v2kMsJPQhBC-djiNq7FpjXmGI2Rd9_Hp6ZBULFNTQ-JATvTzEzT6-Cu2DHW2cZUjeXBFyJ9gI0yuo1YFvYeTLTVtkupKaY2yj2FgBEb7iZcXKUkwXUpvWSBMusju18bqdXFm0cZ6tkTrHMawl0HSkPXSlmLakezL99_1pszZX_h-ZA","text":"AI 勁戰六代是什麼（50字以內）"},"webhookEventId":"01KDS87XH9CYX19SRKZPVYPCWJ","deliveryContext":{"isRedelivery":false},"timestamp":1767152809142,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"edbaa929ef1a4afea84fcbe4185b7b40","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 03:46:53] "POST / HTTP/1.1" 200 -
